In [ ]:
#Don't forget to save your work to your local computer and push to GitHub.
#File -> Download as -> Notebook
#Move file from download folder to local repo clone
#git add filename.ipynb
#git commit -m "put a specific and informative message here of what you worked on"
#git push

In [6]:
import pandas
import os

# This query represents dataset "Concept Set Exploration" for domain "condition" and was generated for All of Us Controlled Tier Dataset v7
dataset_95902282_condition_sql = """
    SELECT
        c_occurrence.person_id,
        c_standard_concept.concept_name as standard_concept_name,
        c_occurrence.condition_start_datetime 
    FROM
        ( SELECT
            * 
        FROM
            `""" + os.environ["WORKSPACE_CDR"] + """.condition_occurrence` c_occurrence 
        WHERE
            (
                condition_concept_id IN  (
                    SELECT
                        DISTINCT c.concept_id 
                    FROM
                        `""" + os.environ["WORKSPACE_CDR"] + """.cb_criteria` c 
                    JOIN
                        (
                            select
                                cast(cr.id as string) as id 
                            FROM
                                `""" + os.environ["WORKSPACE_CDR"] + """.cb_criteria` cr 
                            WHERE
                                concept_id IN (
                                    36712702, 4014295, 432441, 44784550, 45757175, 45757176
                                ) 
                                AND full_text LIKE '%_rank1]%'
                        ) a 
                            ON (
                                c.path LIKE CONCAT('%.',
                            a.id,
                            '.%') 
                            OR c.path LIKE CONCAT('%.',
                            a.id) 
                            OR c.path LIKE CONCAT(a.id,
                            '.%') 
                            OR c.path = a.id) 
                        WHERE
                            is_standard = 1 
                            AND is_selectable = 1
                        )
                )  
                AND (
                    c_occurrence.PERSON_ID IN (
                        SELECT
                            distinct person_id  
                        FROM
                            `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_person` cb_search_person  
                        WHERE
                            cb_search_person.person_id IN (
                                SELECT
                                    criteria.person_id 
                                FROM
                                    (SELECT
                                        DISTINCT person_id,
                                        entry_date,
                                        concept_id 
                                    FROM
                                        `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_all_events` 
                                    WHERE
                                        (
                                            concept_id IN (
                                                SELECT
                                                    DISTINCT c.concept_id 
                                                FROM
                                                    `""" + os.environ["WORKSPACE_CDR"] + """.cb_criteria` c 
                                                JOIN
                                                    (
                                                        select
                                                            cast(cr.id as string) as id 
                                                        FROM
                                                            `""" + os.environ["WORKSPACE_CDR"] + """.cb_criteria` cr 
                                                        WHERE
                                                            concept_id IN (36712702, 45757176, 4014295) 
                                                            AND full_text LIKE '%_rank1]%'
                                                    ) a 
                                                        ON (
                                                            c.path LIKE CONCAT('%.',
                                                        a.id,
                                                        '.%') 
                                                        OR c.path LIKE CONCAT('%.',
                                                        a.id) 
                                                        OR c.path LIKE CONCAT(a.id,
                                                        '.%') 
                                                        OR c.path = a.id) 
                                                    WHERE
                                                        is_standard = 1 
                                                        AND is_selectable = 1
                                                    ) 
                                                    AND is_standard = 1 
                                            )
                                        ) criteria 
                                    ) ))
                        ) c_occurrence 
                    LEFT JOIN
                        `""" + os.environ["WORKSPACE_CDR"] + """.concept` c_standard_concept 
                            ON c_occurrence.condition_concept_id = c_standard_concept.concept_id"""

dataset_95902282_condition_df = pandas.read_gbq(
    dataset_95902282_condition_sql,
    dialect="standard",
    use_bqstorage_api=("BIGQUERY_STORAGE_API_ENABLED" in os.environ),
    progress_bar_type="tqdm_notebook")

dataset_95902282_condition_df.sample(10)

Downloading:   0%|          | 0/390805 [00:00<?, ?rows/s]

,person_id,standard_concept_name,condition_start_datetime
141717,1695593,Second trimester pregnancy,2018-10-18 10:12:00+00:00
41664,1440346,Gestation less than 9 weeks,2021-03-03 00:00:00+00:00
67849,2237397,"Gestation period, 34 weeks",2019-03-05 14:15:27+00:00
198123,1371415,First trimester pregnancy,2022-05-25 00:00:00+00:00
104134,2864629,"Gestation period, 29 weeks",2020-09-08 15:40:00+00:00
377481,1609096,Third trimester pregnancy,2019-08-20 04:00:00+00:00
299794,1691510,Third trimester pregnancy,2018-11-15 05:59:59+00:00
195987,1498440,First trimester pregnancy,2019-04-19 05:00:00+00:00
258667,1148071,Single live birth,2017-06-05 00:00:00+00:00
221958,2589569,First trimester pregnancy,2020-07-22 09:17:00+00:00


In [7]:
#rename dataset 
df = dataset_95902282_condition_df

#view all entries for a specific person_id


person_df = df[df['person_id'] == 5433458]
person_df


#we can see here that we can drop all instances except using Gestation period 40 weeks 
#to calculate gestational age of single live birth

,person_id,standard_concept_name,condition_start_datetime
0,5433458,Gestation less than 20 weeks,2022-01-06 00:00:00+00:00
1,5433458,Gestation less than 20 weeks,2021-12-09 00:00:00+00:00
2,5433458,Gestation less than 20 weeks,2021-11-12 00:00:00+00:00
3,5433458,Gestation less than 20 weeks,2021-12-09 00:00:00+00:00
4,5433458,Gestation less than 20 weeks,2021-11-04 00:00:00+00:00
7361,5433458,"Gestation period, 13 weeks",2021-12-09 00:00:00+00:00
12139,5433458,"Gestation period, 13 weeks",2021-11-12 00:00:00+00:00
13907,5433458,"Gestation period, 36 weeks",2022-04-27 00:00:00+00:00
19783,5433458,"Gestation period, 36 weeks",2022-04-22 00:00:00+00:00
24117,5433458,"Gestation period, 21 weeks",2022-01-06 00:00:00+00:00


In [8]:
#view all entries for standard_concept_name

import pandas as pd

#Create a dictionary to store concept names
concept_dict = {}

#Iterate through column 'standard_concept_name' 
for concept in df['standard_concept_name']:
    if concept in concept_dict:
        concept_dict[concept] += 1
    else:
        concept_dict[concept] = 1

#print nicely
for key, value in concept_dict.items():
    print(f'{key}: {value}')


Gestation less than 20 weeks: 5
Gestation period, 35 weeks: 6198
Gestation period, 18 weeks: 3506
Gestation period, 13 weeks: 2482
Term pregnancy: 4
Gestation period, 36 weeks: 8688
Gestation period, 23 weeks: 2219
Gestation period, 21 weeks: 2622
Gestation period, 38 weeks: 9384
Gestation period, 37 weeks: 9435
Gestation less than 9 weeks: 2109
Gestation period, 25 weeks: 2077
Gestation period, 14 weeks: 1509
Gestation period, 42 weeks: 18
Preterm spontaneous labor with preterm delivery: 503
Gestation period, 12 weeks: 4600
Gestation period, 22 weeks: 2541
Preterm labor in second trimester with preterm delivery in second trimester: 180
Preterm labor with preterm delivery: 79
Gestation period, 16 weeks: 2068
Gestation period, 31 weeks: 3512
Gestation period, 30 weeks: 3415
Gestation period, 19 weeks: 3468
Gestation period, 34 weeks: 5816
Gestation period, 39 weeks: 12126
Gestation period, 15 weeks: 1493
Gestation period, 9 weeks: 39
Gestation period, 10 weeks: 2112
Gestation period, 17

In [9]:
#Create code to calculate conception date from standard_concept_name that gives gestational age

from datetime import timedelta

#make sure condition_start_datetime is formatted correctly
df['condition_start_datetime'] = pd.to_datetime(df['condition_start_datetime'])

# Define a function to calculate the conception date
def calculate_conception(row):
    # Split the 'standard_concept_name' to extract the number of weeks
    concept_name_parts = row['standard_concept_name'].split()
    if len(concept_name_parts) == 4 and concept_name_parts[0] == "Gestation":
        try:
            weeks = int(concept_name_parts[2])
            # Calculate the conception date by subtracting 'weeks' from 'condition_start_datetime'
            conception_date = row['condition_start_datetime'] - timedelta(weeks=weeks)
            return conception_date
        except ValueError:
            pass
    return None

# Apply the function to create the 'conception_date' column
df['conception_date'] = df.apply(calculate_conception, axis=1)

df.sample(10)


,person_id,standard_concept_name,condition_start_datetime,conception_date
2298,1241953,"Gestation period, 35 weeks",2019-05-22 02:47:11+00:00,2018-09-19 02:47:11+00:00
200173,1937017,First trimester pregnancy,2016-06-21 04:00:00+00:00,NaT
235141,1418696,Single live birth,2016-11-22 22:12:28+00:00,NaT
36528,3459227,"Gestation period, 37 weeks",2016-05-12 04:00:00+00:00,2015-08-27 04:00:00+00:00
86695,3065629,"Gestation period, 39 weeks",2019-08-10 20:00:00.765000+00:00,2018-11-10 20:00:00.765000+00:00
54921,7469984,"Gestation period, 12 weeks",2018-07-09 00:00:00+00:00,2018-04-16 00:00:00+00:00
170206,2899024,Second trimester pregnancy,2019-07-22 00:07:42+00:00,NaT
295768,1109919,Third trimester pregnancy,2020-05-20 04:00:00+00:00,NaT
316707,2022273,Third trimester pregnancy,2019-02-22 05:00:00+00:00,NaT
190194,1184729,First trimester pregnancy,2019-11-21 00:00:00+00:00,NaT


In [10]:
#If we revisit our person from above, we will see that we calculated a conception date, but there is some variance.
#Some entries seem wrong (9/9 conception when all the others are august, but others range from 8/6 - 8/18)

person_df = df[df['person_id'] == 5433458]
person_df

person_df.dropna(subset=['conception_date'])

,person_id,standard_concept_name,condition_start_datetime,conception_date
7361,5433458,"Gestation period, 13 weeks",2021-12-09 00:00:00+00:00,2021-09-09 00:00:00+00:00
12139,5433458,"Gestation period, 13 weeks",2021-11-12 00:00:00+00:00,2021-08-13 00:00:00+00:00
13907,5433458,"Gestation period, 36 weeks",2022-04-27 00:00:00+00:00,2021-08-18 00:00:00+00:00
19783,5433458,"Gestation period, 36 weeks",2022-04-22 00:00:00+00:00,2021-08-13 00:00:00+00:00
24117,5433458,"Gestation period, 21 weeks",2022-01-06 00:00:00+00:00,2021-08-12 00:00:00+00:00
29836,5433458,"Gestation period, 38 weeks",2022-05-05 00:00:00+00:00,2021-08-12 00:00:00+00:00
31302,5433458,"Gestation period, 38 weeks",2022-05-09 00:00:00+00:00,2021-08-16 00:00:00+00:00
53741,5433458,"Gestation period, 12 weeks",2021-11-04 00:00:00+00:00,2021-08-12 00:00:00+00:00
62933,5433458,"Gestation period, 30 weeks",2022-03-09 00:00:00+00:00,2021-08-11 00:00:00+00:00
72081,5433458,"Gestation period, 34 weeks",2022-04-07 00:00:00+00:00,2021-08-12 00:00:00+00:00


In [11]:
#The earlier the ultrasound, the more accurate (before 14 weeks, and especially at 8 weeks). American College of Obstetricians and Gynecologists. 
#8 weeks is a common time in US healthcare to get US and measurements, and we have n = 9944.
#If we can't calculate single live birth from the 8 week entry, then let's choose 10, 12, 13, weeks etc and prioritize
#which entries are used to calculate.

#I'm not yet sure how to handle instances like the person above who the first two lines are both week 13, but at appointments 
#4 weeks apart; one is clearly wrong. fortunately we would use her week 8 data anyway....

In [22]:
#Let's reduce data by dropping columns that aren't used to calculate conception date 
#or are Single live birth or preterm birth terms
include_list = ["Single live birth", "Term pregnancy", "Preterm spontaneous labor with preterm delivery"]
include_list.append("Preterm labor in second trimester with preterm delivery in second trimester")
include_list.append("Preterm labor with preterm delivery")
include_list.append("Preterm labor in third trimester with preterm delivery in third trimester")

# Condition to filter rows with 'conception_date' as NaT but keep conditions in include_list
condition = (df['conception_date'].notna()) | (df['standard_concept_name'].isin(include_list))

# Apply the condition to filter the DataFrame
filtered_df = df[condition]

# Reset the index if needed
# filtered_df.reset_index(drop=True, inplace=True)

filtered_df.sample(10)

,person_id,standard_concept_name,condition_start_datetime,conception_date
91353,3154850,"Gestation period, 10 weeks",2019-12-27 00:00:00+00:00,2019-10-18 00:00:00+00:00
98730,3004784,"Gestation period, 33 weeks",2019-02-02 00:47:41+00:00,2018-06-16 00:47:41+00:00
40446,2875616,"Gestation period, 37 weeks",2019-06-28 17:02:06+00:00,2018-10-12 17:02:06+00:00
109948,1046688,"Gestation period, 20 weeks",2020-02-07 16:00:00+00:00,2019-09-20 16:00:00+00:00
80605,2869247,"Gestation period, 39 weeks",2018-07-02 00:00:00+00:00,2017-10-02 00:00:00+00:00
86626,5422302,"Gestation period, 39 weeks",2016-09-19 23:02:48+00:00,2015-12-21 23:02:48+00:00
127254,3185860,"Gestation period, 26 weeks",2019-09-03 04:00:00+00:00,2019-03-05 04:00:00+00:00
126610,2101309,"Gestation period, 26 weeks",2016-08-26 04:00:00+00:00,2016-02-26 04:00:00+00:00
264927,1824198,Single live birth,1998-12-27 06:00:00+00:00,NaT
270684,3116826,Single live birth,2007-11-14 05:00:00+00:00,NaT


In [24]:
#Create a df to calculate gestational_age_at_birth, using the filtered df that excluded gestational dates that we aren't using.

#Function to format age in weeks and days 
def format_weeks_and_days(days):
    weeks, remaining_days = divmod(days, 7)
    return f"{weeks}w{remaining_days}d"


# Filter the dataframe again to keep only 'Single live birth' entries
single_live_birth_df = filtered_df[filtered_df['standard_concept_name'] == "Single live birth"]

# Merge 'condition_start_datetime' from 'Single live birth' rows with the filtered dataframe based on 'person_id'
filtered_df2 = filtered_df.merge(single_live_birth_df[['person_id', 'condition_start_datetime']], on='person_id', how='left', suffixes=('', '_birth'))

#Make sure columns are in correct format
filtered_df2['conception_date'] = pd.to_datetime(filtered_df2['conception_date'])
filtered_df2['condition_start_datetime_birth'] = pd.to_datetime(filtered_df2['condition_start_datetime_birth'])


# Calculate 'gestational_age_at_birth' by subtracting 'condition_start_datetime_birth' from 'conception_date'
filtered_df2['gestational_age_at_birth'] = (filtered_df2['condition_start_datetime_birth'] - filtered_df2['conception_date']).dt.days
filtered_df2['gestational_age_at_birth'] = filtered_df2['gestational_age_at_birth'].apply(format_weeks_and_days)


filtered_df2.sample(20)



,person_id,standard_concept_name,condition_start_datetime,conception_date,condition_start_datetime_birth,gestational_age_at_birth
383988,1926230,"Gestation period, 29 weeks",2020-04-04 22:23:00+00:00,2019-09-14 22:23:00+00:00,2020-06-18 09:32:00+00:00,39.0w4.0d
708692,1657714,"Gestation period, 8 weeks",2020-02-18 05:00:00+00:00,2019-12-24 05:00:00+00:00,2020-09-24 05:00:00+00:00,39.0w2.0d
602631,1376030,Single live birth,2017-10-02 14:09:00+00:00,NaT,2019-10-11 15:35:00+00:00,nanwnand
588204,6944502,Single live birth,1998-10-13 06:00:00+00:00,NaT,1998-10-13 06:00:00+00:00,nanwnand
51779,2466595,"Gestation period, 36 weeks",2022-01-10 21:10:00+00:00,2021-05-03 21:10:00+00:00,2020-02-12 05:00:00+00:00,-64.0w1.0d
678753,2799170,Single live birth,2012-07-18 00:00:00+00:00,NaT,2014-01-19 00:00:00+00:00,nanwnand
263571,1530704,"Gestation period, 34 weeks",2020-06-03 18:50:00+00:00,2019-10-09 18:50:00+00:00,2009-04-10 04:00:00+00:00,-548.0w1.0d
312676,1629895,"Gestation period, 39 weeks",2020-12-08 05:00:00+00:00,2020-03-10 05:00:00+00:00,2020-12-16 20:31:00+00:00,40.0w1.0d
675674,2076126,Single live birth,2021-11-09 01:27:18+00:00,NaT,2021-11-09 01:27:18+00:00,nanwnand
242520,3072300,"Gestation period, 31 weeks",2020-09-04 00:00:00+00:00,2020-01-31 00:00:00+00:00,2020-09-24 00:00:00+00:00,33.0w6.0d


In [28]:
# Merge 'filtered_df' with the original DataFrame 'data' based on 'person_id'
merged_df = filtered_df2.merge(filtered_df, on='person_id', how='left', suffixes=('', '_original'))

# Display the merged DataFrame
merged_df.sample(20)

,person_id,standard_concept_name,condition_start_datetime,conception_date,condition_start_datetime_birth,gestational_age_at_birth,standard_concept_name_original,condition_start_datetime_original,conception_date_original
24253822,1766820,Single live birth,2020-02-18 00:00:00+00:00,NaT,2022-01-17 00:00:00+00:00,nanwnand,"Gestation period, 41 weeks",2020-02-18 00:00:00+00:00,2019-05-07 00:00:00+00:00
18116422,1595943,"Gestation period, 20 weeks",2021-10-18 00:00:00+00:00,2021-05-31 00:00:00+00:00,2022-01-26 00:00:00+00:00,34.0w2.0d,"Gestation period, 33 weeks",2019-09-07 00:00:00+00:00,2019-01-19 00:00:00+00:00
3199294,2357912,"Gestation period, 36 weeks",2018-10-04 05:00:00+00:00,2018-01-25 05:00:00+00:00,2018-10-12 05:00:00+00:00,37.0w1.0d,"Gestation period, 32 weeks",2018-09-05 05:00:00+00:00,2018-01-24 05:00:00+00:00
17275154,2381629,"Gestation period, 27 weeks",2019-03-07 05:00:00+00:00,2018-08-30 05:00:00+00:00,2021-05-28 05:00:00+00:00,143.0w1.0d,"Gestation period, 11 weeks",2018-11-02 05:00:00+00:00,2018-08-17 05:00:00+00:00
28085868,1405558,"Gestation period, 8 weeks",2019-03-19 05:00:00+00:00,2019-01-22 05:00:00+00:00,2019-11-11 05:00:00+00:00,41.0w6.0d,Single live birth,2019-11-12 05:00:00+00:00,NaT
12460829,1578549,"Gestation period, 19 weeks",2018-10-04 16:00:00+00:00,2018-05-24 16:00:00+00:00,2019-02-14 02:44:00+00:00,37.0w6.0d,"Gestation period, 23 weeks",2020-07-09 05:59:59+00:00,2020-01-30 05:59:59+00:00
6276280,1221480,"Gestation period, 37 weeks",2018-07-31 00:00:00+00:00,2017-11-14 00:00:00+00:00,2018-08-16 00:00:00+00:00,39.0w2.0d,Single live birth,2021-03-26 00:00:00+00:00,NaT
18899472,1927720,"Gestation period, 40 weeks",2019-12-14 05:00:00+00:00,2019-03-09 05:00:00+00:00,2019-12-14 05:00:00+00:00,40.0w0.0d,"Gestation period, 36 weeks",2019-11-15 05:00:00+00:00,2019-03-08 05:00:00+00:00
9174490,1244224,"Gestation period, 12 weeks",2017-05-30 05:00:00+00:00,2017-03-07 05:00:00+00:00,2017-12-19 05:00:00+00:00,41.0w0.0d,"Gestation period, 8 weeks",2020-09-17 05:00:00+00:00,2020-07-23 05:00:00+00:00
16892008,1868289,"Gestation period, 29 weeks",2016-03-01 00:00:00+00:00,2015-08-11 00:00:00+00:00,2021-06-21 00:00:00+00:00,305.0w6.0d,"Gestation period, 11 weeks",2015-10-21 00:00:00+00:00,2015-08-05 00:00:00+00:00
